<a href="https://colab.research.google.com/github/yeesem/Advanced-Tensorflow-Specialization/blob/main/Fully_CNN_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [20]:
import os
import zipfile
import PIL.Image,PIL.ImageFont,PIL.ImageDraw
import numpy as np

import tensorflow as tf
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
import seaborn as sns

# Download the Dataset

In [21]:
# download the dataset (zipped file)
!wget https://storage.googleapis.com/learning-datasets/fcnn-dataset.zip  -O /tmp/fcnn-dataset.zip

--2024-06-11 11:47:42--  https://storage.googleapis.com/learning-datasets/fcnn-dataset.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.8.207, 142.251.170.207, 173.194.174.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.8.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 125577577 (120M) [application/zip]
Saving to: ‘/tmp/fcnn-dataset.zip’

/tmp/fcnn-dataset.z 100%[===================>] 119.76M  29.5MB/s    in 4.9s    

2024-06-11 11:47:47 (24.4 MB/s) - ‘/tmp/fcnn-dataset.zip’ saved [125577577/125577577]



In [22]:
local_zip = '/tmp/fcnn-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip,'r')
zip_ref.extractall('/tmp/fcnn')
zip_ref.close()

In [23]:
# pixel labels in the video frames
class_names = ['sky', 'building','column/pole', 'road', 'side walk', 'vegetation', 'traffic light', 'fence', 'vehicle', 'pedestrian', 'byciclist', 'void']

# Load and Prepare the Dataset

In [24]:
def map_filename_to_image_and_mask(t_filename,a_filename,height = 224,width = 224):
  # Convert image and mask files to tensor
  img_raw = tf.io.read_file(t_filename)
  anno_raw = tf.io.read_file(a_filename)
  image = tf.image.decode_png(img_raw)
  annotation = tf.image.decode_png(anno_raw)

  # Resize image and segmentation mask
  image = tf.image.resize(image,(height,width,))
  annotation = tf.image.resize(annotation,(height,width,))
  image = tf.reshape(image,(height,width,3,))
  annotation = tf.cast(annotation,dtype = tf.int32)
  annotation = tf.reshape(annotation,(height,width,1,))
  stack_list = []

  # Reshape segmentation masks
  for c in range(len(class_names)):
    # mask = tf.equal(annotation[:,:,0], tf.constant(c)) is creating a binary mask where each element
    # in the mask is True if the corresponding element in annotation[:,:,0] is equal to the class c,
    # and False otherwise.
    # tf.equal(annotation[:,:,0], tf.constant(c)): This function compares each element in the 2D array
    # annotation[:,:,0] with the constant c. It returns a new 2D tensor of the same shape where each element
    # is True if the corresponding element in annotation[:,:,0] is equal to c, and False otherwise.
    mask = tf.equal(annotation[:,:,0],tf.constant(c))
    stack_list.append(tf.cast(mask,dtype = tf.int32))

  annotation = tf.stack(stack_list,axis = 2)

  # Normalize pixels in the input image
  image = image / 127.5
  # shift pixel value
  # This line shifts the scaled pixel values to the range [-1, 1].
  image -= 1

  return image,annotation

In [25]:
# show folders inside the dataset you downloaded
!ls /tmp/fcnn/dataset1

annotations_prepped_test  annotations_prepped_train  images_prepped_test  images_prepped_train


In [26]:
BATCH_SIZE = 64

def get_dataset_slice_paths(image_dir,label_map_dir):
  image_file_list = os.listdir(image_dir)
  label_map_file_list = os.listdir(label_map_dir)
  image_paths = [os.path.join(image_dir,fname) for fname in image_file_list]
  label_map_paths = [os.path.join(label_map_dir,fname) for fname in label_map_file_list]

  return image_paths,label_map_paths

def get_training_dataset(image_paths,label_map_paths):
  training_dataset = tf.data.Dataset.from_tensor_slices((image_paths,label_map_paths))
  training_dataset = training_dataset.map(map_filename_to_image_and_mask)
  training_dataset = training_dataset.shuffle(100,reshuffle_each_iteration = True)
  training_dataset = training_dataset.batch(BATCH_SIZE)
  # repeat(): This method makes the dataset repeat indefinitely. This is useful for training loops
  # where you want the dataset to be available continuously without needing to be re-initialized.
  training_dataset = training_dataset.repeat()
  # repeat(): This method makes the dataset repeat indefinitely. This is useful for training loops where you want the
  # dataset to be available continuously without needing to be re-initialized.
  training_dataset = training_dataset.prefetch(-1)

  return training_dataset

def get_validation_dataset(image_paths,label_map_paths):
  validation_dataset = tf.data.Dataset.from_tensor_slices((image_paths,label_map_paths))
  validation_dataset = validation_dataset.map(map_filename_to_image_and_mask)
  validation_dataset = validation_dataset.batch(BATCH_SIZE)
  validation_dataset = validation_dataset.repeat()

  return validation_dataset

# Dataset Visualization

In [28]:
# get the paths to the images
training_image_paths, training_label_map_paths = get_dataset_slice_paths('/tmp/fcnn/dataset1/images_prepped_train/','/tmp/fcnn/dataset1/annotations_prepped_train/')
validation_image_paths, validation_label_map_paths = get_dataset_slice_paths('/tmp/fcnn/dataset1/images_prepped_test/','/tmp/fcnn/dataset1/annotations_prepped_test/')

# Generate the train and val set
training_dataset = get_training_dataset(training_image_paths,training_label_map_paths)
validation_dataset = get_validation_dataset(validation_image_paths,validation_label_map_paths)

In [29]:
# Generate a list that contains one color for each class
colors = sns.color_palette(None,len(class_names))

# print class name - normalized RGB tuple pairs
# the tuple values will be multiplied by 255 in the helper functions later
# to convert to the (0,0,0) to (255,255,255) RGB values you might be familiar with
for class_name,color in zip(class_names,colors):
  print(f"P{class_name} -- {color}")

Psky -- (0.12156862745098039, 0.4666666666666667, 0.7058823529411765)
Pbuilding -- (1.0, 0.4980392156862745, 0.054901960784313725)
Pcolumn/pole -- (0.17254901960784313, 0.6274509803921569, 0.17254901960784313)
Proad -- (0.8392156862745098, 0.15294117647058825, 0.1568627450980392)
Pside walk -- (0.5803921568627451, 0.403921568627451, 0.7411764705882353)
Pvegetation -- (0.5490196078431373, 0.33725490196078434, 0.29411764705882354)
Ptraffic light -- (0.8901960784313725, 0.4666666666666667, 0.7607843137254902)
Pfence -- (0.4980392156862745, 0.4980392156862745, 0.4980392156862745)
Pvehicle -- (0.7372549019607844, 0.7411764705882353, 0.13333333333333333)
Ppedestrian -- (0.09019607843137255, 0.7450980392156863, 0.8117647058823529)
Pbyciclist -- (0.12156862745098039, 0.4666666666666667, 0.7058823529411765)
Pvoid -- (1.0, 0.4980392156862745, 0.054901960784313725)
